# [1] 실습 환경 준비

In [33]:
!pip install opencv-python

In [34]:
import cv2
import os
import glob
import shutil
import random
import string
import numpy as np

In [35]:
PATH_DEFECT = 'dataset/Defect_images/'
PATH_MASK = 'dataset/Mask_images/'
PATH_NODEFECT = 'dataset/NODefect_images/'

## Dataset 불러오기

In [36]:
random.seed(0)

defect_list = glob.glob(PATH_DEFECT + '*.png')
mask_list = glob.glob(PATH_MASK + '*.png')
pass_list = glob.glob(PATH_NODEFECT + '**/*.png')

# Match defect-mask pairs
new_defect_list = list()
new_mask_list = list()
for defect in defect_list:
    num = defect.split('/')[-1].split('_')[0]
    for mask in mask_list:
        num_mask = mask.split('/')[-1].split('_')[0]
        if num == num_mask:
            new_defect_list.append(defect)
            new_mask_list.append(mask)
            break
defect_list = new_defect_list
mask_list = new_mask_list

## 첫 발송 데이터 생성

In [37]:
# The first dataset given
if os.path.exists('dataset/1') is False:
    os.mkdir('dataset/1')
for file_name in pass_list + defect_list:
    if random.randint(0, 9) < 2:
        barcode = ''.join(random.choices(string.ascii_letters + string.digits, k=16))
        shutil.copy2(file_name, 'dataset/1/' + barcode + '.png')

## 두번째 데이터 생성

In [38]:
# The second dataset
if os.path.exists('dataset/2') is False:
    os.mkdir('dataset/2')
if os.path.exists('dataset/2/OK') is False:
    os.mkdir('dataset/2/OK')
if os.path.exists('dataset/2/FAIL') is False:
    os.mkdir('dataset/2/FAIL')
idx = 0
for file_name in pass_list:
    img = cv2.imread(file_name)
    height, width, _ = img.shape
    step = height // 2

    for i in range(width // step):
        w = i * step
        if w < width - height and random.randint(0, 9) < 2:
            patch = img[:, w:w+height, :]
            cv2.imwrite('dataset/2/OK/%04d.png' % idx, patch)
            idx += 1 

patch_list = list()
for item in zip(defect_list, mask_list):
    defect, mask = item

    img_d = cv2.imread(defect)
    img_m = cv2.imread(mask)

    height, width, _ = img_d.shape
    step = height // 2
    for i in range(width // step):
        w = i * step
        if w < width - height:
            patch = img_d[:, w:w+height, :]
            patch_d = img_m[:, w:w+height, :]
            if patch_d.sum() > 0:
                patch_list.append(patch)

random.shuffle(patch_list)
patch_list_fraction = patch_list[:len(patch_list)//3]
for idx, patch in enumerate(patch_list_fraction):
    cv2.imwrite('dataset/2/FAIL/%04d.png' % idx, patch)

## 세번째 데이터 생성

In [39]:
# The third dataset
if os.path.exists('dataset/3') is False:
    os.mkdir('dataset/3')
if os.path.exists('dataset/3/OK') is False:
    os.mkdir('dataset/3/OK')
if os.path.exists('dataset/3/FAIL') is False:
    os.mkdir('dataset/3/FAIL')
if os.path.exists('dataset/3/MASK') is False:
    os.mkdir('dataset/3/MASK')
idx = 0
for file_name in pass_list:
    img = cv2.imread(file_name)
    height, width, _ = img.shape
    step = height // 2

    for i in range(width // step):
        w = i * step
        if w < width - height and random.randint(0, 9) < 3:
            patch = img[:, w:w+height, :]
            cv2.imwrite('dataset/3/OK/%04d.png' % idx, patch)
            idx += 1 

patch_pair_list = list()
for item in zip(defect_list, mask_list):
    defect, mask = item

    img_d = cv2.imread(defect)
    img_m = cv2.imread(mask)

    height, width, _ = img_d.shape
    step = height // 2
    for i in range(width // step):
        w = i * step
        if w < width - height:
            patch = img_d[:, w:w+height, :]
            patch_d = img_m[:, w:w+height, :]

            if patch_d.sum() > 0:
                patch_pair_list.append((patch, patch_d))

random.shuffle(patch_pair_list)
for idx, pair in enumerate(patch_pair_list):
    patch, patch_d = pair
    cv2.imwrite('dataset/3/FAIL/%04d.png' % idx, patch)
    cv2.imwrite('dataset/3/MASK/%04d.png' % idx, patch_d)

## 실전 데이터 생성

In [40]:
# The test dataset
if os.path.exists('dataset/input_data') is False:
    os.mkdir('dataset/input_data')
if os.path.exists('dataset/output_csv') is False:
    os.mkdir('dataset/output_csv')
    
idx = 0
for file_name in pass_list:
    img = cv2.imread(file_name)
    height, width, _ = img.shape
    step = height // 2

    for i in range(width // step):
        w = i * step
        if w < width - height and random.randint(0, 9) < 5:
            patch = img[:, w:w+height, :]
            cv2.imwrite('dataset/input_data/ok_%04d.png' % idx, patch)
            idx += 1 

patch_pair_list = list()
for item in zip(defect_list, mask_list):
    defect, mask = item

    img_d = cv2.imread(defect)
    img_m = cv2.imread(mask)

    height, width, _ = img_d.shape
    step = height // 2
    for i in range(width // step):
        w = i * step
        if w < width - height:
            patch = img_d[:, w:w+height, :]
            patch_d = img_m[:, w:w+height, :]

            if patch_d.sum() > 0:
                patch_pair_list.append((patch, patch_d))

random.shuffle(patch_pair_list)
for idx, pair in enumerate(patch_pair_list):
    patch, patch_d = pair
    cv2.imwrite('dataset/input_data/fail_%04d.png' % idx, patch)

# [2] 실행 가능성 확인하기

In [41]:
import glob
import os
import tensorflow as tf
import cv2
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from tensorflow.keras.models import Sequential

## 하이퍼파라미터

In [42]:
## 단순한 모델 설정
EPOCHS = 10

DATASET_PATH = 'dataset/2/'
DATASET_OK_PATTERN = DATASET_PATH + 'OK/*.png'
DATASET_FAIL_PATTERN = DATASET_PATH + 'FAIL/*.png'

RESULT_SAVE_PATH = 'results/'

## 단순한 모델 설정

In [43]:
def Model():
    return Sequential([Conv2D(32, (3, 3), activation='relu', input_shape=(256, 256, 1)), #tensorflow 버전업으로 코드 변경
                       MaxPool2D(),
                       Conv2D(64, (3, 3), activation='relu'),
                       MaxPool2D(),
                       Conv2D(128, (3, 3), activation='relu'),
                       MaxPool2D(),
                       Conv2D(256, (3, 3), activation='relu'),
                       MaxPool2D(),
                       Flatten(),
                       Dense(1, activation='sigmoid')])## 데이터셋 불러오기

## 데이터셋 불러오기

In [44]:
def preprocess(file_name):
    img = tf.io.read_file(file_name)
    img = tf.image.decode_png(img, channels=1) #tensorflow 버전업으로 코드 변경
    return tf.image.convert_image_dtype(img, tf.float32)

In [45]:
ok_list = glob.glob(DATASET_OK_PATTERN)
ds_ok = tf.data.Dataset.list_files(ok_list)
ds_ok_label = tf.data.Dataset.from_tensor_slices([0] * len(ok_list))

ds_ok = ds_ok.map(preprocess)
ds_ok = tf.data.Dataset.zip((ds_ok, ds_ok_label))

fail_list = glob.glob(DATASET_FAIL_PATTERN)
ds_fail = tf.data.Dataset.list_files(fail_list)
ds_fail_label = tf.data.Dataset.from_tensor_slices([1] * len(fail_list))

ds_fail = ds_fail.map(preprocess)
ds_fail = tf.data.Dataset.zip((ds_fail, ds_fail_label))## Train, Valid 데이터셋 나누기

ds = tf.data.Dataset.concatenate(ds_ok, ds_fail)

## Train, Valid 데이터셋 나누기

In [46]:
ds_size = len(ok_list) + len(fail_list)
train_size = int(ds_size * 0.7)

ds = ds.shuffle(ds_size)
ds_train = ds.take(train_size).shuffle(1024, reshuffle_each_iteration=True).batch(32)
ds_valid = ds.skip(train_size).batch(32)

## 모델 생성 및 학습

In [47]:
model = Model()
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(ds_train, validation_data=ds_valid, epochs=EPOCHS)

In [49]:
def mkdir(path):
    if os.path.exists(path) is False:
        os.mkdir(path)

mkdir(RESULT_SAVE_PATH)
mkdir(RESULT_SAVE_PATH + '/TP')
mkdir(RESULT_SAVE_PATH + '/TN')
mkdir(RESULT_SAVE_PATH + '/FP')
mkdir(RESULT_SAVE_PATH + '/FN')

index = 0
for imgs, labels in ds_valid:
    preds = model.predict(imgs)
    for idx in range(imgs.shape[0]):
        gt = labels[idx].numpy()
        y = preds[idx]
        
        if gt == 1 and y > 0.5:
            path = RESULT_SAVE_PATH + '/TP'
        elif gt == 1 and y <= 0.5:
            path = RESULT_SAVE_PATH + '/FN'
        elif gt == 0 and y > 0.5:
            path = RESULT_SAVE_PATH + '/FP'
        else:
            path = RESULT_SAVE_PATH + '/TN'
            
        cv2.imwrite(path + '/%.4f_%04d.png' % (y, index), imgs[idx].numpy() * 255)
        index +=1

# [3] 데이터 정리하기

# TFRecord Builder

In [ ]:
import glob
import os
import tensorflow as tf
import cv2

## Paths and Hyperparameters

In [ ]:
DATASET_OK_PATTERN = 'dataset/3/OK/*.png'
DATASET_FAIL_PATTERN = 'dataset/3/FAIL/*.png'

TFRECORD_PATH = 'tfrecords/'
IMAGE_PER_TFRECORD = 100

## Import data

In [ ]:
ok_list = glob.glob(DATASET_OK_PATTERN)
fail_list = glob.glob(DATASET_FAIL_PATTERN)

num_ok = len(ok_list)
num_fail = len(fail_list)

# Oversampling
fail_list_new = list()
for _ in range(num_ok // num_fail):
    fail_list_new += fail_list
fail_list_new += fail_list[: num_ok % num_fail]
fail_list = fail_list_new

ok_label = [0] * len(ok_list)
fail_label = [1] * len(fail_list)

file_list = ok_list + fail_list
label_list = ok_label + fail_label

## TFRecord functions

In [ ]:
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def image_example(image_string, label):
    image_shape = tf.image.decode_image(image_string).shape

    feature = {
        'height': _int64_feature(image_shape[0]),
        'width': _int64_feature(image_shape[1]),
        'depth': _int64_feature(image_shape[2]),
        'label': _int64_feature(label),
        'image_raw': _bytes_feature(image_string),
    }

    return tf.train.Example(features=tf.train.Features(feature=feature))

## Write TFRecords

In [ ]:
if os.path.exists(TFRECORD_PATH) is False:
    os.mkdir(TFRECORD_PATH)

num_tfrecords = len(file_list) // IMAGE_PER_TFRECORD
if len(file_list) % IMAGE_PER_TFRECORD != 0:
    num_tfrecords += 1

for idx in range(num_tfrecords):
    idx0 = idx * IMAGE_PER_TFRECORD
    idx1 = idx0 + IMAGE_PER_TFRECORD
    record_file = TFRECORD_PATH + '%05d.tfrecords' % idx
    with tf.io.TFRecordWriter(record_file) as writer:
        for filename, label in zip(file_list[idx0:idx1],
                                   label_list[idx0:idx1]):
            image_string = open(filename, 'rb').read()
            tf_example = image_example(image_string, label)
            writer.write(tf_example.SerializeToString())

# [4] 모델 학습 및 검증하기

In [50]:
!pip install tensorflow_addons

In [51]:
!pip install matplotlib

In [52]:
import glob
import os
import tensorflow as tf
import tensorflow_addons as tfa #안되면 pip install 
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.layers import Conv2D, MaxPool2D, Concatenate, Flatten, Dense

## 하이퍼파라미터

In [53]:
EPOCHS = 3
RESULT_SAVE_PATH = 'results/'

## Inception-based 모델 정의

In [54]:
def Model():
    def inception(filters):
        def subnetwork(x):
            h1 = Conv2D(filters, (1, 1), padding='same', activation='relu')(x)
            h1 = MaxPool2D()(h1)
            
            h2 = Conv2D(filters // 2, (1, 1), padding='same', activation='relu')(x)
            h2 = Conv2D(filters, (3, 3), padding='same', activation='relu')(h2)
            h2 = MaxPool2D()(h2)
            
            h3 = Conv2D(filters // 2, (1, 1), padding='same', activation='relu')(x)
            h3 = Conv2D(filters, (5, 5), padding='same', activation='relu')(h3)
            h3 = MaxPool2D()(h3)
            return Concatenate()([h1, h2, h3])
        return subnetwork
    
    x = tf.keras.Input(shape=(256, 256, 3))
    h = inception(16)(x)
    h = inception(32)(h)
    h = inception(32)(h)
    h = inception(32)(h)
    h = inception(32)(h)
    h = Flatten()(h)
    h = Dense(1024, activation='relu')(h)
    y = Dense(1, activation='sigmoid')(h)
    return tf.keras.Model(inputs=x, outputs=y)

## Data 전처리 함수 정의

In [55]:
def preprocess(img):
    return tf.image.convert_image_dtype(img, tf.float32)

## Data Augmentation 함수 정의

In [56]:
def augmentation(img, label):
    def flip(x):
        x = tf.image.random_flip_left_right(x)
        x = tf.image.random_flip_up_down(x)
        return x
    
    def rotate(x):
        x = tf.cond(tf.random.uniform(shape=[], minval=0.0, maxval=1.0, dtype=tf.float32) > 0.5,
                   lambda: tfa.image.rotate(x,
                                       tf.random.uniform(shape=[], minval=0.0, maxval=360.0, dtype=tf.float32),
                                       interpolation='BILINEAR'),
                   lambda: x)
        return x
    
    def translation(x):
        dx = tf.random.uniform(shape=[], minval=-10.0, maxval=10.0, dtype=tf.float32)
        dy = tf.random.uniform(shape=[], minval=-10.0, maxval=10.0, dtype=tf.float32)
        x = tf.cond(tf.random.uniform(shape=[], minval=0.0, maxval=1.0, dtype=tf.float32) > 0.5,
                    lambda: tfa.image.transform(x,
                                                [0, 0, dx, 0, 0, dy, 0, 0],
                                                interpolation='BILINEAR'),
                    lambda: x)
        return x
    
    img = flip(img)
    img = rotate(img)
    img = translation(img)
           
    return img, label

## TFRecords 불러오기

In [57]:
tffiles = glob.glob('tfrecords/*')
raw_image_dataset = tf.data.TFRecordDataset(tffiles)

image_feature_description = {
    'height': tf.io.FixedLenFeature([], tf.int64),
    'width': tf.io.FixedLenFeature([], tf.int64),
    'depth': tf.io.FixedLenFeature([], tf.int64),
    'label': tf.io.FixedLenFeature([], tf.int64),
    'image_raw': tf.io.FixedLenFeature([], tf.string),
}

def _parse_image_function(example_proto):
    return tf.io.parse_single_example(example_proto, image_feature_description)

def _parse_image_label(parsed_dataset):
    return preprocess(tf.image.decode_png(parsed_dataset['image_raw'])), parsed_dataset['label']

parsed_image_dataset = raw_image_dataset.map(_parse_image_function)
dataset = parsed_image_dataset.map(_parse_image_label)

## 데이터셋 나누기

In [58]:
ds_size = 0
for _ in dataset:
    ds_size += 1

train_size = int(ds_size * 0.7)

ds = dataset.shuffle(ds_size)
ds_train = ds.take(train_size).shuffle(1024, reshuffle_each_iteration=True).prefetch(1024).batch(32).map(augmentation)
ds_valid = ds.skip(train_size).prefetch(1024).batch(32)

## 모델 생성

In [59]:
model = Model()
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

## 모델 학습하기

In [ ]:
earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=30, verbose=1)
history = model.fit(ds_train,
                    validation_data=ds_valid,
                    epochs=EPOCHS,
                    callbacks=[earlystopping])

## 학습 결과 Plot

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure()
plt.plot(loss, 'ro-')
plt.plot(val_loss, 'bo-')
plt.ylabel('Cross Entropy')
plt.xlabel('Epoch')
plt.title('Training and Validation Loss')
plt.show()

## 모델 저장

In [ ]:
model.save('model/inception_model.h5')

# [5] 프로그램 전달하기

# 배치형 동작 프로그램

In [61]:
import tensorflow as tf
from datetime import datetime
import time
import glob

## 하이퍼파라미터, Path

In [62]:
THRES_LEVEL = 0.5 #0.5이하면 ok / 1에 가까울 수록 fail

INPUT_PATH = 'dataset/input_data/'
CSV_PATH = 'dataset/output_csv/'

## 모델 불러오기

In [63]:
model = tf.keras.models.load_model('model/inception_model.h5')

## 입력 데이터 전처리

In [64]:
def preprocess(file_name):
    img = tf.io.read_file(file_name)
    img = tf.image.decode_image(img)
    return tf.image.convert_image_dtype(img, tf.float32)

## 입력 데이터 불러오기

In [65]:
file_list = glob.glob(INPUT_PATH + '*.png')
dataset = tf.data.Dataset.list_files(file_list).map(preprocess)

## 알고리즘 구동 및 CSV 결과 저장

In [66]:
now = datetime.now().strftime('%Y%d%m_%H%M%S')
with open(CSV_PATH + now + '.csv', 'w') as f:
    for image, filename in zip(dataset, file_list):
        image = image[tf.newaxis, ...] # HWC -> NHWC
        
        a = time.time()
        predict = model.predict(image)[0][0]
        print('Inference Time:', time.time() - a)
        
        if predict > THRES_LEVEL:
            label = 'FAIL'
        else:
            label = 'OK'
        
        f.write(','.join([filename, label, str(predict)]) + '\n')

Inference Time: 0.3540363311767578
Inference Time: 0.09731197357177734
Inference Time: 0.11469411849975586
Inference Time: 0.13763189315795898
Inference Time: 0.12566208839416504
Inference Time: 0.12965631484985352
Inference Time: 0.11269617080688477
Inference Time: 0.11967778205871582
Inference Time: 0.12318587303161621
Inference Time: 0.12744998931884766
Inference Time: 0.12466549873352051
Inference Time: 0.12766742706298828
Inference Time: 0.12167477607727051
Inference Time: 0.16419267654418945
Inference Time: 0.1795179843902588
Inference Time: 0.1436145305633545
Inference Time: 0.1436152458190918
Inference Time: 0.14660930633544922
Inference Time: 0.15358948707580566
Inference Time: 0.18103480339050293
Inference Time: 0.2194216251373291
Inference Time: 0.14760422706604004
Inference Time: 0.18602609634399414
Inference Time: 0.2492361068725586
Inference Time: 0.2528502941131592
Inference Time: 0.15059423446655273
Inference Time: 0.18202996253967285
Inference Time: 0.226393461227417
I

Inference Time: 0.11619901657104492
Inference Time: 0.1242666244506836
Inference Time: 0.11469149589538574
Inference Time: 0.11670064926147461
Inference Time: 0.1326436996459961
Inference Time: 0.11253976821899414
Inference Time: 0.11320638656616211
Inference Time: 0.13364434242248535
Inference Time: 0.1466045379638672
Inference Time: 0.1236717700958252
Inference Time: 0.1296539306640625
Inference Time: 0.1356353759765625
Inference Time: 0.14261889457702637
Inference Time: 0.12466549873352051
Inference Time: 0.12467265129089355
Inference Time: 0.11768460273742676
Inference Time: 0.13065218925476074
Inference Time: 0.1186835765838623
Inference Time: 0.13364338874816895
Inference Time: 0.1296520233154297
Inference Time: 0.12820005416870117
Inference Time: 0.12666058540344238
Inference Time: 0.1376330852508545
Inference Time: 0.11967897415161133
Inference Time: 0.13913774490356445
Inference Time: 0.11681365966796875
Inference Time: 0.11220765113830566
Inference Time: 0.13065099716186523
I

Inference Time: 0.11867976188659668
Inference Time: 0.12865900993347168
Inference Time: 0.12366843223571777
Inference Time: 0.133650541305542
Inference Time: 0.15159368515014648
Inference Time: 0.1371462345123291
Inference Time: 0.14760231971740723
Inference Time: 0.14316725730895996
Inference Time: 0.14162230491638184
Inference Time: 0.14761066436767578
Inference Time: 0.13466596603393555
Inference Time: 0.16209697723388672
Inference Time: 0.14162135124206543
Inference Time: 0.176530122756958
Inference Time: 0.15957331657409668
Inference Time: 0.12366890907287598
Inference Time: 0.12765765190124512
Inference Time: 0.1401360034942627
Inference Time: 0.15459656715393066
Inference Time: 0.13364267349243164
Inference Time: 0.16606616973876953
Inference Time: 0.13217616081237793
Inference Time: 0.16555571556091309
Inference Time: 0.13962340354919434
Inference Time: 0.12366819381713867
Inference Time: 0.11967897415161133
Inference Time: 0.19547653198242188
Inference Time: 0.1880791187286377

Inference Time: 0.12318873405456543
Inference Time: 0.13715529441833496
Inference Time: 0.1406242847442627
Inference Time: 0.12067890167236328
Inference Time: 0.11668539047241211
Inference Time: 0.12067580223083496
Inference Time: 0.13663315773010254
Inference Time: 0.1386251449584961
Inference Time: 0.12865543365478516
Inference Time: 0.15387558937072754
Inference Time: 0.1216726303100586
Inference Time: 0.13064837455749512
Inference Time: 0.1296532154083252
Inference Time: 0.12417960166931152
Inference Time: 0.13313961029052734
Inference Time: 0.10970544815063477
Inference Time: 0.13164472579956055
Inference Time: 0.2641618251800537
Inference Time: 0.11073946952819824
Inference Time: 0.10402631759643555
Inference Time: 0.1057133674621582
Inference Time: 0.11170578002929688
Inference Time: 0.10870957374572754
Inference Time: 0.10671663284301758
Inference Time: 0.10571527481079102
Inference Time: 0.11668586730957031
Inference Time: 0.12518000602722168
Inference Time: 0.1142082214355468

Inference Time: 0.13914012908935547
Inference Time: 0.13220977783203125
Inference Time: 0.15658044815063477
Inference Time: 0.12366867065429688
Inference Time: 0.12816739082336426
Inference Time: 0.13109350204467773
Inference Time: 0.1156911849975586
Inference Time: 0.11867976188659668
Inference Time: 0.11967897415161133
Inference Time: 0.14162158966064453
Inference Time: 0.15059542655944824
Inference Time: 0.12566375732421875
Inference Time: 0.1216738224029541
Inference Time: 0.1176900863647461
Inference Time: 0.11668753623962402
Inference Time: 0.13405537605285645
Inference Time: 0.11469173431396484
Inference Time: 0.13067984580993652
Inference Time: 0.12218594551086426
Inference Time: 0.12865543365478516
Inference Time: 0.20046281814575195
Inference Time: 0.16954755783081055
Inference Time: 0.12466645240783691
Inference Time: 0.12367033958435059
Inference Time: 0.13515019416809082
Inference Time: 0.14561128616333008
Inference Time: 0.1276566982269287
Inference Time: 0.12666106224060

Inference Time: 0.12267088890075684
Inference Time: 0.1132512092590332
Inference Time: 0.12366914749145508
Inference Time: 0.11070442199707031
Inference Time: 0.11868166923522949
Inference Time: 0.11269807815551758
Inference Time: 0.12067937850952148
Inference Time: 0.10870862007141113
Inference Time: 0.11868166923522949
Inference Time: 0.11967182159423828
Inference Time: 0.10771036148071289
Inference Time: 0.12366604804992676
Inference Time: 0.1211848258972168
Inference Time: 0.11070489883422852
Inference Time: 0.10970497131347656
Inference Time: 0.1107025146484375
Inference Time: 0.12217569351196289
Inference Time: 0.1181936264038086
Inference Time: 0.11420941352844238
Inference Time: 0.10870814323425293
Inference Time: 0.12666010856628418
Inference Time: 0.10970878601074219
Inference Time: 0.12267088890075684
Inference Time: 0.10970664024353027
Inference Time: 0.10771012306213379
Inference Time: 0.12566065788269043
Inference Time: 0.12418222427368164
Inference Time: 0.10971212387084

Inference Time: 0.12167477607727051
Inference Time: 0.11192846298217773
Inference Time: 0.11768293380737305
Inference Time: 0.10970592498779297
Inference Time: 0.12666058540344238
Inference Time: 0.14162182807922363
Inference Time: 0.19447731971740723
Inference Time: 0.1356363296508789
Inference Time: 0.157578706741333
Inference Time: 0.11016130447387695
Inference Time: 0.12665772438049316
Inference Time: 0.12067651748657227
Inference Time: 0.1301875114440918
Inference Time: 0.12267112731933594
Inference Time: 0.124664306640625
Inference Time: 0.12167596817016602
Inference Time: 0.1515960693359375
Inference Time: 0.11668539047241211
Inference Time: 0.11668634414672852
Inference Time: 0.1107032299041748
Inference Time: 0.10970592498779297
Inference Time: 0.12067651748657227
Inference Time: 0.1107034683227539
Inference Time: 0.11322593688964844
Inference Time: 0.12069201469421387
Inference Time: 0.10821938514709473
Inference Time: 0.12067532539367676
Inference Time: 0.11170029640197754
I

Inference Time: 0.1575760841369629
Inference Time: 0.13663554191589355
Inference Time: 0.12127876281738281
Inference Time: 0.12842726707458496
Inference Time: 0.13814210891723633
Inference Time: 0.12865591049194336
Inference Time: 0.11768436431884766
Inference Time: 0.11668896675109863
Inference Time: 0.18451142311096191
Inference Time: 0.1356348991394043
Inference Time: 0.11768221855163574
Inference Time: 0.1261746883392334
Inference Time: 0.1266627311706543
Inference Time: 0.13663578033447266
Inference Time: 0.11784243583679199
Inference Time: 0.12568330764770508
Inference Time: 0.14712071418762207
Inference Time: 0.12365865707397461
Inference Time: 0.1296525001525879
Inference Time: 0.1376338005065918
Inference Time: 0.17846417427062988
Inference Time: 0.1296532154083252
Inference Time: 0.13463950157165527
Inference Time: 0.14861226081848145
Inference Time: 0.12166953086853027
Inference Time: 0.13065481185913086
Inference Time: 0.14013385772705078
Inference Time: 0.14760375022888184

Inference Time: 0.11169981956481934
Inference Time: 0.1186826229095459
Inference Time: 0.11768841743469238
Inference Time: 0.11569023132324219
Inference Time: 0.11919236183166504
Inference Time: 0.11868762969970703
Inference Time: 0.12566351890563965
Inference Time: 0.11967802047729492
Inference Time: 0.11001944541931152
Inference Time: 0.1118006706237793
Inference Time: 0.12135124206542969
Inference Time: 0.11038064956665039
Inference Time: 0.11469149589538574
Inference Time: 0.12189221382141113
Inference Time: 0.11321377754211426
Inference Time: 0.13343596458435059
Inference Time: 0.12218379974365234
Inference Time: 0.12865495681762695
Inference Time: 0.13164710998535156
Inference Time: 0.13463997840881348
Inference Time: 0.13464021682739258
Inference Time: 0.1406240463256836
Inference Time: 0.12067627906799316
Inference Time: 0.12865447998046875
Inference Time: 0.11569499969482422
Inference Time: 0.12366795539855957
Inference Time: 0.10970520973205566
Inference Time: 0.1092169284820

Inference Time: 0.11420536041259766
Inference Time: 0.1132044792175293
Inference Time: 0.14162015914916992
Inference Time: 0.12566280364990234
Inference Time: 0.12923550605773926
Inference Time: 0.11437034606933594
Inference Time: 0.14064931869506836
Inference Time: 0.11967706680297852
Inference Time: 0.1107029914855957
Inference Time: 0.11868119239807129
Inference Time: 0.14860272407531738
Inference Time: 0.12366771697998047
Inference Time: 0.13164734840393066
Inference Time: 0.11269998550415039
Inference Time: 0.1595747470855713
Inference Time: 0.1675574779510498
Inference Time: 0.1396341323852539
Inference Time: 0.13115978240966797
Inference Time: 0.13563823699951172
Inference Time: 0.21342825889587402
Inference Time: 0.1685476303100586
Inference Time: 0.1436150074005127
Inference Time: 0.1266627311706543
Inference Time: 0.15309977531433105
